## Init and import

This depends on a new custom module in concordia
Run "pip install --editable .[dev]" in your environment to install the new build if you haven't

In [1]:
# @title Imports
import sys
import os
path = os.path.abspath('..')
if path not in sys.path:
    sys.path.insert(0, path)
import datetime

from concordia import components as generic_components
from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia.agents import basic_agent
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import measurements as measurements_lib
from concordia.utils import html as html_lib
from concordia.utils import plotting

import logging
logging.basicConfig(level=logging.ERROR, filename='components_testing.log')
logger = logging.getLogger('huggingface')

In [2]:
# Setup sentence encoder
from sentence_transformers import SentenceTransformer
st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
embedder = st5_model.encode

In [49]:
# from concordia.language_model import huggingface_model
import huggingface_model
import importlib
importlib.reload(huggingface_model)
# ===== MODEL LIST =====
# bigscience/bloom-560m
# mistralai/Mixtral-8x7B-v0.1 (not tested yet)
# mistralai/Mistral-7B-v0.1
# (check huggingface for more)

model = huggingface_model.HuggingFaceLanguageModel(
    model_name='bigscience/bloom-560m',
    logits=True,
)

Using device: cuda:0


In [4]:
#@title Make the clock
time_step = datetime.timedelta(minutes=20)
SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[time_step, datetime.timedelta(seconds=10)])


In [5]:
measurements = measurements_lib.Measurements()

In [54]:
import custom_components.logits_query_metric as lm
import importlib
importlib.reload(lm)

context = """
Below is a statement. If it is true, answer "yes". If it is false, answer "no".
"""

quiz_metric = lm.LogitsQueryMetric(model, 
                                   "Alice", 
                                   clock, 
                                   "./custom_components/truth_or_lie.json", 
                                   measurements=measurements, 
                                   verbose=True,
                                   query=["yes", "no"]) # query should match up with order of options in the question json
quiz_metric.observe(context)

To the question 1 + 1 = 2, the agent answered "no", the probs are {'yes': 0.2863, 'no': 0.7137}
To the question Red is a color, the agent answered "no", the probs are {'yes': 0.1309, 'no': 0.8691}
To the question Earth is flat, the agent answered "no", the probs are {'yes': 0.001, 'no': 0.999}
To the question There are 365 days in a year, the agent answered "no", the probs are {'yes': 0.01, 'no': 0.99}
To the question There aren't 365 days in a year, the agent answered "no", the probs are {'yes': 0.0104, 'no': 0.9896}
To the question The sum of the angles of a triangle is 180 degrees, the agent answered "no", the probs are {'yes': 0.0094, 'no': 0.9906}
To the question The printing press was invented in the 13th century, the agent answered "no", the probs are {'yes': 0.138, 'no': 0.862}
To the question The melting point of carbon is around 3500 degrees Celsius, the agent answered "no", the probs are {'yes': 0.0398, 'no': 0.9602}
LogitQueryScores of Alice: Agent completed the quiz. 3 out

In [12]:
import importlib
import examples.custom_components.logits_query_metric as lm
importlib.reload(lm)

results = quiz_metric.get_results()
import csv
# Create a csv file with the results if not already created
if not os.path.exists('./results/'):
    os.makedirs('./results/')
with open('./results/logits.txt', 'w') as file:
    # Write Question: Response, and space after each pair
    writer = csv.writer(file)
    for result in results:
        writer.writerow(result)
    file.close()